In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification
from tqdm import tqdm

c:\Users\ducli\miniconda3\envs\bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Đọc tập dữ liệu từ file CSV
df = pd.read_csv(r"..\data\processed\merged_reviews.csv")

C:\Users\ducli\AppData\Local\Temp\ipykernel_13624\3623922201.py:2: DtypeWarning: Columns (28,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"..\data\processed\merged_reviews.csv")


In [3]:
df.columns

Index(['comment', 'orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid',
       'shopid', 'rating_star', 'status', 'mtime', 'editable', 'opt', 'filter',
       'mentioned', 'is_hidden', 'can_follow_up', 'follow_up', 'submit_time',
       'author_username', 'author_portrait', 'author_shopid', 'anonymous',
       'images', 'videos', 'product_items', 'delete_reason', 'delete_operator',
       'ItemRatingReply', 'tags', 'editable_date', 'show_reply', 'like_count',
       'liked', 'sync_to_social', 'detailed_rating',
       'exclude_scoring_due_low_logistic', 'loyalty_info', 'template_tags',
       'has_template_tag', 'sync_to_social_toggle', 'sip_info',
       'is_repeated_purchase', 'display_variation_filter', 'overall_fit',
       'is_normal_item', 'viewed', 'show_view', 'sync_to_social_detail',
       'profile', 'size_info_tags', 'size_info_abt', 'image_data', 'key_media',
       'is_super_review', 'super_reviewer_tag', 'is_newly_created',
       'template_hints', 'template_tags_hin

In [4]:
df['rating'].unique()

array([ 1,  0, -1], dtype=int64)

In [5]:
df['rating_star'].unique()

array([5, 4, 3, 1, 2], dtype=int64)

In [6]:
df['date'] = pd.to_datetime(df['ctime'], unit='s')

In [7]:
df = df[df['date'] < '2024-04-11']

In [8]:
minimal_field = ['comment', 'itemid', 'date', 'cmtid','rating_star']
df = df[minimal_field]

In [9]:
df.dropna(inplace=True)

In [1]:
# Chọn thiết bị GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tải mô hình và tokenizer từ Hugging Face Model Hub
model_name = "mr4/phobert-base-vi-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = RobertaForSequenceClassification.from_pretrained(model_name).to(device)

# Hàm để phân tích cảm xúc của một đoạn văn bản
def analyze_sentiment(texts):
    # Chuyển đổi đầu vào thành danh sách nếu không phải là danh sách
    if isinstance(texts, str):
        texts = [texts]
    elif isinstance(texts, float):
        texts = [str(texts)]  # Chuyển số thành chuỗi

    labels = []
    probabilities = []
    for single_text in texts:
        inputs = tokenizer(single_text, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        probabilities.append(torch.softmax(logits, dim=1).tolist()[0])
        labels.append(torch.argmax(logits, dim=1).tolist()[0])

    return labels, probabilities



c:\Users\ducli\miniconda3\envs\bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Tạo các cột mới cho Predicted Label và Probabilities
df['predicted_label'] = None
df['probabilities'] = None

# Sử dụng hàm để phân tích cảm xúc của các đoạn văn bản trong tập dữ liệu đã lọc
for idx, row in tqdm(df.iterrows(), total=len(df)):
    text = row["comment"]
    labels, probabilities = analyze_sentiment(text)

    # Gán kết quả dự đoán vào các cột tương ứng
    df.at[idx, 'predicted_label'] = labels[0]
    df.at[idx, 'probabilities'] = probabilities[0]

# In ra một số dòng đầu tiên của DataFrame để kiểm tra
print(df.head())


100%|██████████| 2075125/2075125 [6:55:08<00:00, 83.31it/s]   

                                             comment       itemid  \
0  giao hàng nhanh, đóng gói cẩn thận lịch sự , n...  10000420589   
1  Sản phẩm giống như giới thiệu, được gói cẩn th...  10000420589   
2  Mùi hương:trà xanh 🍵\nCông dụng:làm sạch răng ...  10000420589   
3  sản phẩm quá quen thuộc rồi ! mùi hương cũng ổ...  10000420589   
4  Công dụng:kem đánh răng\nMùi hương:thơm\n\nMới...  10000420589   

                  date        cmtid  rating_star predicted_label  \
0  2022-04-05 09:51:32   7966754081            5               1   
1  2022-05-25 12:25:24   8476029460            5               1   
2  2023-09-11 11:01:04  12548411626            5               1   
3  2022-04-20 17:09:38   8134845040            5               1   
4  2022-10-26 12:14:16   9907636426            5               1   

                                       probabilities  
0  [0.0017404258251190186, 0.9929191470146179, 0....  
1  [0.0018135074060410261, 0.9928566217422485, 0....  
2  [0.00204

In [3]:
df.to_csv(r"..\data\processed\merged_reviews_minimal_with_bert.csv", index=False)